# Tests for Amphibole functions

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("example_table.csv")

In [4]:
df['Fe2O3'] = 0

In [8]:
df

,SiO2,TiO2,Al2O3,FeO,MnO,MgO,CaO,Na2O,K2O,Fe2O3
0,43.55,1.28,13.62,9.62,0.00,14.53,10.25,2.51,0.64,0
1,52.50,0.00,4.40,20.40,0.30,18.00,0.50,0.10,0.00,0
2,49.01,0.20,14.91,0.55,0.00,28.62,0.23,1.46,0.00,0
3,40.80,0.00,18.30,25.00,0.40,11.40,0.20,1.60,0.00,0
4,38.41,0.48,19.72,35.53,2.30,0.03,0.03,1.16,0.04,0
5,56.78,0.00,0.96,13.31,0.33,24.71,0.83,0.12,0.00,0
6,52.50,0.00,1.60,27.30,0.80,14.70,0.60,0.10,0.00,0
7,49.55,0.00,0.12,40.14,1.84,4.89,0.70,0.00,0.00,0
8,50.60,0.00,0.04,36.10,5.80,6.40,0.40,0.05,0.00,0
9,55.27,0.00,0.34,4.52,16.62,19.18,1.19,0.26,0.00,0


In [18]:
df2 = df.set_axis(['px', 'px2', 'px3', 'px4', 'px5', 'px6', 'px7', 'px8', 'px9', 'px10'], axis=0, inplace=False)
df2

,SiO2,TiO2,Al2O3,FeO,MnO,MgO,CaO,Na2O,K2O,Fe2O3
px,43.55,1.28,13.62,9.62,0.00,14.53,10.25,2.51,0.64,0
px2,52.50,0.00,4.40,20.40,0.30,18.00,0.50,0.10,0.00,0
px3,49.01,0.20,14.91,0.55,0.00,28.62,0.23,1.46,0.00,0
px4,40.80,0.00,18.30,25.00,0.40,11.40,0.20,1.60,0.00,0
px5,38.41,0.48,19.72,35.53,2.30,0.03,0.03,1.16,0.04,0
px6,56.78,0.00,0.96,13.31,0.33,24.71,0.83,0.12,0.00,0
px7,52.50,0.00,1.60,27.30,0.80,14.70,0.60,0.10,0.00,0
px8,49.55,0.00,0.12,40.14,1.84,4.89,0.70,0.00,0.00,0
px9,50.60,0.00,0.04,36.10,5.80,6.40,0.40,0.05,0.00,0
px10,55.27,0.00,0.34,4.52,16.62,19.18,1.19,0.26,0.00,0


In [19]:
df2.T

,px,px2,px3,px4,px5,px6,px7,px8,px9,px10
SiO2,43.55,52.5,49.01,40.8,38.41,56.78,52.5,49.55,50.60,55.27
TiO2,1.28,0.0,0.20,0.0,0.48,0.00,0.0,0.00,0.00,0.00
Al2O3,13.62,4.4,14.91,18.3,19.72,0.96,1.6,0.12,0.04,0.34
FeO,9.62,20.4,0.55,25.0,35.53,13.31,27.3,40.14,36.10,4.52
MnO,0.00,0.3,0.00,0.4,2.30,0.33,0.8,1.84,5.80,16.62
MgO,14.53,18.0,28.62,11.4,0.03,24.71,14.7,4.89,6.40,19.18
CaO,10.25,0.5,0.23,0.2,0.03,0.83,0.6,0.70,0.40,1.19
Na2O,2.51,0.1,1.46,1.6,1.16,0.12,0.1,0.00,0.05,0.26
K2O,0.64,0.0,0.00,0.0,0.04,0.00,0.0,0.00,0.00,0.00
Fe2O3,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00


In [20]:
df2 = df.copy(deep=True)
df2.loc[2,"SiO2"] = 500
df2

,SiO2,TiO2,Al2O3,FeO,MnO,MgO,CaO,Na2O,K2O,Fe2O3
0,43.55,1.28,13.62,9.62,0.00,14.53,10.25,2.51,0.64,0
1,52.50,0.00,4.40,20.40,0.30,18.00,0.50,0.10,0.00,0
2,500.00,0.20,14.91,0.55,0.00,28.62,0.23,1.46,0.00,0
3,40.80,0.00,18.30,25.00,0.40,11.40,0.20,1.60,0.00,0
4,38.41,0.48,19.72,35.53,2.30,0.03,0.03,1.16,0.04,0
5,56.78,0.00,0.96,13.31,0.33,24.71,0.83,0.12,0.00,0
6,52.50,0.00,1.60,27.30,0.80,14.70,0.60,0.10,0.00,0
7,49.55,0.00,0.12,40.14,1.84,4.89,0.70,0.00,0.00,0
8,50.60,0.00,0.04,36.10,5.80,6.40,0.40,0.05,0.00,0
9,55.27,0.00,0.34,4.52,16.62,19.18,1.19,0.26,0.00,0


In [ ]:
from Amphibole import normalization


In [ ]:
normalized_dataset = normalization(df)

In [ ]:
# dataset with elements normalized, must add some other calculations
normalized_dataset